In [428]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn import linear_model
from sklearn.metrics import mean_absolute_error,mean_squared_error
from sklearn import preprocessing, svm
from sklearn.model_selection import train_test_split

In [429]:
dfRAWLoadHistory = pd.read_excel("dataset.xlsx", sheet_name="load_history",parse_dates= {"date" : ["year","month","day"]})
dfRAWTempHistory = pd.read_excel("dataset.xlsx", sheet_name="temperature_history",parse_dates= {"date" : ["year","month","day"]})

In [430]:
dfRAWTempHistory.head()

,date,obsn,station_id,h1,h2,h3,h4,h5,h6,h7,...,h18,h19,h20,h21,h22,h23,h24,Avg Temp,High,Low
0,2015-01-01,1,1,45.0,41.0,40.0,39.0,43.0,44.0,41.0,...,62.0,59.0,59.0,57.0,56.0,54.0,50.0,50.125000,63.0,37.0
1,2015-01-01,2,2,55.0,55.0,54.0,50.0,48.0,44.0,42.0,...,55.0,49.0,49.0,48.0,47.0,49.0,50.0,53.333333,64.0,42.0
2,2015-01-01,3,3,47.0,45.0,44.0,39.0,39.0,38.0,37.0,...,61.0,56.0,55.0,55.0,54.0,55.0,55.0,50.875000,65.0,37.0
3,2015-01-01,4,4,53.0,51.0,49.0,45.0,43.0,41.0,39.0,...,57.0,54.0,44.0,43.0,39.0,39.0,39.0,48.708333,61.0,39.0
4,2015-01-01,5,5,54.0,52.0,50.0,46.0,42.0,41.0,39.0,...,60.0,55.0,52.0,49.0,46.0,48.0,51.0,53.000000,69.0,38.0


In [431]:
dfRAWLoadHistory.head()

,date,obsn,zone_id,Holiday,Weekend,h1,h2,h3,h4,h5,...,h15,h16,h17,h18,h19,h20,h21,h22,h23,h24
0,2015-01-01,1,1.0,1,0,66025.0,64655.0,63898.0,64078.0,65734.0,...,50102.0,49364.0,51966.0,60480.0,64995.0,65275.0,65167.0,65055.0,65449.0,61156.0
1,2015-01-01,2,2.0,1,0,16908.0,16505.0,16572.0,16928.0,17119.0,...,13573.0,13193.0,14185.0,16864.0,18205.0,18290.0,17980.0,16959.0,16217.0,14805.0
2,2015-01-01,3,3.0,1,0,126314.0,123368.0,119247.0,117562.0,118398.0,...,127738.0,126884.0,135218.0,155491.0,157905.0,156895.0,153409.0,147031.0,137768.0,128078.0
3,2015-01-01,4,4.0,1,0,136288.0,133110.0,128663.0,126846.0,127747.0,...,137825.0,136904.0,145897.0,167770.0,170376.0,169285.0,165525.0,158642.0,148647.0,138193.0
4,2015-01-01,5,5.0,1,0,539.0,512.0,505.0,503.0,499.0,...,581.0,562.0,615.0,674.0,748.0,718.0,692.0,660.0,605.0,577.0


In [432]:
dfRAWLoadHistory.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26256 entries, 0 to 26255
Data columns (total 29 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   date     26256 non-null  datetime64[ns]
 1   obsn     26241 non-null  object        
 2   zone_id  24615 non-null  float64       
 3   Holiday  26256 non-null  int64         
 4   Weekend  26256 non-null  int64         
 5   h1       25232 non-null  float64       
 6   h2       25232 non-null  float64       
 7   h3       25232 non-null  float64       
 8   h4       25232 non-null  float64       
 9   h5       25232 non-null  float64       
 10  h6       25232 non-null  float64       
 11  h7       25232 non-null  float64       
 12  h8       25232 non-null  float64       
 13  h9       25232 non-null  float64       
 14  h10      25232 non-null  float64       
 15  h11      25232 non-null  float64       
 16  h12      25232 non-null  float64       
 17  h13      25232 non-null  float6

In [433]:
dfRAWLoadHistory = dfRAWLoadHistory.replace('', np.nan)

# using dictionary to convert specific columns
convert_dict = {'Holiday': bool, 'Weekend': bool, 'zone_id': str}
dfRAWLoadHistory = dfRAWLoadHistory.astype(convert_dict)
dfRAWLoadHistory.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26256 entries, 0 to 26255
Data columns (total 29 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   date     26256 non-null  datetime64[ns]
 1   obsn     26241 non-null  object        
 2   zone_id  26256 non-null  object        
 3   Holiday  26256 non-null  bool          
 4   Weekend  26256 non-null  bool          
 5   h1       25232 non-null  float64       
 6   h2       25232 non-null  float64       
 7   h3       25232 non-null  float64       
 8   h4       25232 non-null  float64       
 9   h5       25232 non-null  float64       
 10  h6       25232 non-null  float64       
 11  h7       25232 non-null  float64       
 12  h8       25232 non-null  float64       
 13  h9       25232 non-null  float64       
 14  h10      25232 non-null  float64       
 15  h11      25232 non-null  float64       
 16  h12      25232 non-null  float64       
 17  h13      25232 non-null  float6

In [434]:
dfTotalLoadHistory = dfRAWLoadHistory[(dfRAWLoadHistory.obsn == "total")]
dfTotalLoadHistory.drop(columns='zone_id',inplace = True)
dfTotalLoadHistory.head()

/var/folders/_4/65xymvxn5xgb3xsst5lm02800000gn/T/ipykernel_92252/1559307592.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfTotalLoadHistory.drop(columns='zone_id',inplace = True)


,date,obsn,Holiday,Weekend,h1,h2,h3,h4,h5,h6,...,h15,h16,h17,h18,h19,h20,h21,h22,h23,h24
15,2015-01-01,total,True,False,976022.0,940929.0,911360.0,903748.0,926770.0,950478.0,...,931515.0,922874.0,978192.0,1128920.0,1163657.0,1162872.0,1144546.0,1104189.0,1054152.0,982493.0
31,2015-01-02,total,False,False,936473.0,919261.0,908778.0,911527.0,929152.0,988469.0,...,1069949.0,1042032.0,1079327.0,1198336.0,1227602.0,1217351.0,1193901.0,1145354.0,1085611.0,1001548.0
47,2015-01-03,total,False,True,951182.0,909672.0,898850.0,884750.0,900030.0,923800.0,...,922920.0,912623.0,946551.0,1046284.0,1069254.0,1043129.0,1018950.0,972760.0,918042.0,844958.0
63,2015-01-04,total,False,True,771658.0,738315.0,720847.0,709735.0,715169.0,724002.0,...,943851.0,953043.0,1006350.0,1107198.0,1127991.0,1110790.0,1096348.0,1046516.0,968805.0,888023.0
79,2015-01-05,total,False,False,845034.0,828042.0,830246.0,849710.0,875595.0,975527.0,...,1125570.0,1132154.0,1176300.0,1288604.0,1330396.0,1320305.0,1286228.0,1229536.0,1145582.0,1058491.0


In [435]:
dfLoadHistoryToPred = dfRAWLoadHistory[(dfRAWLoadHistory.obsn != "total")&(pd.isna(dfRAWLoadHistory['h1']))]
dfLoadHistoryToPred.drop(columns='obsn',inplace = True)
dfLoadHistoryToPred.head()

/var/folders/_4/65xymvxn5xgb3xsst5lm02800000gn/T/ipykernel_92252/3364734008.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfLoadHistoryToPred.drop(columns='obsn',inplace = True)


,date,zone_id,Holiday,Weekend,h1,h2,h3,h4,h5,h6,...,h15,h16,h17,h18,h19,h20,h21,h22,h23,h24
6784,2016-02-29,1.0,False,False,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6785,2016-02-29,2.0,False,False,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6786,2016-02-29,3.0,False,False,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6787,2016-02-29,4.0,False,False,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6788,2016-02-29,5.0,False,False,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [436]:
dfLoadHistoryToPred.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 960 entries, 6784 to 26254
Data columns (total 28 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   date     960 non-null    datetime64[ns]
 1   zone_id  960 non-null    object        
 2   Holiday  960 non-null    bool          
 3   Weekend  960 non-null    bool          
 4   h1       0 non-null      float64       
 5   h2       0 non-null      float64       
 6   h3       0 non-null      float64       
 7   h4       0 non-null      float64       
 8   h5       0 non-null      float64       
 9   h6       0 non-null      float64       
 10  h7       0 non-null      float64       
 11  h8       0 non-null      float64       
 12  h9       0 non-null      float64       
 13  h10      0 non-null      float64       
 14  h11      0 non-null      float64       
 15  h12      0 non-null      float64       
 16  h13      0 non-null      float64       
 17  h14      0 non-null      float

In [437]:
#create df of data without missing/to pred values 

dfLoadHistory = dfRAWLoadHistory[(dfRAWLoadHistory.obsn != "total")&(~dfRAWLoadHistory['h1'].isnull())]
dfLoadHistory.head(20)

,date,obsn,zone_id,Holiday,Weekend,h1,h2,h3,h4,h5,...,h15,h16,h17,h18,h19,h20,h21,h22,h23,h24
0,2015-01-01,1,1.0,True,False,66025.0,64655.0,63898.0,64078.0,65734.0,...,50102.0,49364.0,51966.0,60480.0,64995.0,65275.0,65167.0,65055.0,65449.0,61156.0
1,2015-01-01,2,2.0,True,False,16908.0,16505.0,16572.0,16928.0,17119.0,...,13573.0,13193.0,14185.0,16864.0,18205.0,18290.0,17980.0,16959.0,16217.0,14805.0
2,2015-01-01,3,3.0,True,False,126314.0,123368.0,119247.0,117562.0,118398.0,...,127738.0,126884.0,135218.0,155491.0,157905.0,156895.0,153409.0,147031.0,137768.0,128078.0
3,2015-01-01,4,4.0,True,False,136288.0,133110.0,128663.0,126846.0,127747.0,...,137825.0,136904.0,145897.0,167770.0,170376.0,169285.0,165525.0,158642.0,148647.0,138193.0
4,2015-01-01,5,5.0,True,False,539.0,512.0,505.0,503.0,499.0,...,581.0,562.0,615.0,674.0,748.0,718.0,692.0,660.0,605.0,577.0
5,2015-01-01,6,6.0,True,False,6884.0,6651.0,6580.0,6709.0,7032.0,...,6913.0,6690.0,7275.0,8713.0,9234.0,9160.0,8826.0,8277.0,7360.0,6391.0
6,2015-01-01,7,7.0,True,False,133143.0,129964.0,125772.0,124217.0,125375.0,...,134595.0,133520.0,142438.0,164148.0,167084.0,166000.0,162181.0,155252.0,145073.0,134415.0
7,2015-01-01,8,8.0,True,False,136288.0,133110.0,128663.0,126846.0,127747.0,...,137825.0,136904.0,145897.0,167770.0,170376.0,169285.0,165525.0,158642.0,148647.0,138193.0
8,2015-01-01,9,9.0,True,False,3179.0,3011.0,3008.0,2969.0,3276.0,...,3829.0,3808.0,3958.0,4403.0,4663.0,4587.0,4327.0,3959.0,3582.0,3243.0
9,2015-01-01,10,10.0,True,False,75298.0,67423.0,64105.0,63916.0,75907.0,...,65722.0,65680.0,64987.0,66184.0,65176.0,64966.0,64966.0,65407.0,73660.0,76537.0


In [438]:
dfRAWTempHistory.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14760 entries, 0 to 14759
Data columns (total 30 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   date        14760 non-null  datetime64[ns]
 1   obsn        14760 non-null  int64         
 2   station_id  14760 non-null  int64         
 3   h1          14697 non-null  float64       
 4   h2          14697 non-null  float64       
 5   h3          14697 non-null  float64       
 6   h4          14697 non-null  float64       
 7   h5          14697 non-null  float64       
 8   h6          14697 non-null  float64       
 9   h7          14697 non-null  float64       
 10  h8          14697 non-null  float64       
 11  h9          14697 non-null  float64       
 12  h10         14697 non-null  float64       
 13  h11         14697 non-null  float64       
 14  h12         14697 non-null  float64       
 15  h13         14697 non-null  float64       
 16  h14         14697 non-

In [439]:
dfRAWTempHistory = dfRAWTempHistory.replace('', np.nan)

# using dictionary to convert specific columns
convert_dict = {'station_id':str}
dfRAWTempHistory = dfRAWTempHistory.astype(convert_dict)
dfRAWTempHistory.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14760 entries, 0 to 14759
Data columns (total 30 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   date        14760 non-null  datetime64[ns]
 1   obsn        14760 non-null  int64         
 2   station_id  14760 non-null  object        
 3   h1          14697 non-null  float64       
 4   h2          14697 non-null  float64       
 5   h3          14697 non-null  float64       
 6   h4          14697 non-null  float64       
 7   h5          14697 non-null  float64       
 8   h6          14697 non-null  float64       
 9   h7          14697 non-null  float64       
 10  h8          14697 non-null  float64       
 11  h9          14697 non-null  float64       
 12  h10         14697 non-null  float64       
 13  h11         14697 non-null  float64       
 14  h12         14697 non-null  float64       
 15  h13         14697 non-null  float64       
 16  h14         14697 non-

In [440]:
dfTempHistoryToPred = dfRAWTempHistory[(pd.isna(dfRAWTempHistory['h1']))]
dfTempHistoryToPred.drop(columns='obsn',inplace = True)
dfTempHistoryToPred.head()

/var/folders/_4/65xymvxn5xgb3xsst5lm02800000gn/T/ipykernel_92252/1393011075.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfTempHistoryToPred.drop(columns='obsn',inplace = True)


,date,station_id,h1,h2,h3,h4,h5,h6,h7,h8,...,h18,h19,h20,h21,h22,h23,h24,Avg Temp,High,Low
14697,2019-06-23,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14698,2019-06-23,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14699,2019-06-23,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14700,2019-06-23,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14701,2019-06-23,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [441]:
dfTempHist=dfRAWTempHistory[(~dfRAWTempHistory['h1'].isnull())]
dfTempHist.isnull().sum()

date          0
obsn          0
station_id    0
h1            0
h2            0
h3            0
h4            0
h5            0
h6            0
h7            0
h8            0
h9            0
h10           0
h11           0
h12           0
h13           0
h14           0
h15           0
h16           0
h17           0
h18           0
h19           0
h20           0
h21           0
h22           0
h23           0
h24           0
Avg Temp      0
High          0
Low           0
dtype: int64

In [442]:
dfTempHist.head(20)

,date,obsn,station_id,h1,h2,h3,h4,h5,h6,h7,...,h18,h19,h20,h21,h22,h23,h24,Avg Temp,High,Low
0,2015-01-01,1,1,45.0,41.0,40.0,39.0,43.0,44.0,41.0,...,62.0,59.0,59.0,57.0,56.0,54.0,50.0,50.125000,63.0,37.0
1,2015-01-01,2,2,55.0,55.0,54.0,50.0,48.0,44.0,42.0,...,55.0,49.0,49.0,48.0,47.0,49.0,50.0,53.333333,64.0,42.0
2,2015-01-01,3,3,47.0,45.0,44.0,39.0,39.0,38.0,37.0,...,61.0,56.0,55.0,55.0,54.0,55.0,55.0,50.875000,65.0,37.0
3,2015-01-01,4,4,53.0,51.0,49.0,45.0,43.0,41.0,39.0,...,57.0,54.0,44.0,43.0,39.0,39.0,39.0,48.708333,61.0,39.0
4,2015-01-01,5,5,54.0,52.0,50.0,46.0,42.0,41.0,39.0,...,60.0,55.0,52.0,49.0,46.0,48.0,51.0,53.000000,69.0,38.0
5,2015-01-01,6,6,51.0,51.0,49.0,48.0,49.0,50.0,47.0,...,56.0,48.0,48.0,43.0,42.0,40.0,42.0,50.750000,61.0,40.0
6,2015-01-01,7,7,53.0,52.0,51.0,47.0,47.0,46.0,45.0,...,62.0,55.0,53.0,54.0,52.0,52.0,54.0,55.625000,69.0,45.0
7,2015-01-01,8,8,54.0,53.0,50.0,49.0,44.0,44.0,42.0,...,60.0,55.0,51.0,49.0,47.0,50.0,50.0,52.875000,69.0,39.0
8,2015-01-01,9,9,52.0,53.0,51.0,43.0,39.0,44.0,43.0,...,62.0,52.0,46.0,45.0,45.0,52.0,53.0,53.375000,71.0,39.0
9,2015-01-02,10,1,49.0,48.0,51.0,51.0,54.0,55.0,55.0,...,65.0,62.0,62.0,60.0,55.0,59.0,59.0,58.083333,68.0,48.0


In [443]:
# Above is left with multiple DFs
#dfRAWLoadHistory - RAW from excel
#dfRAWTempHistory - RAW from excel

#dfTempHistory - Temp Data
#dfLoadHistory - Energy Load Data

#dfLoadHistoryToPred - blank values need to Predict
#dfTempHistoryToPred - blank values need to Predict

#dfTotalLoadHistory - Daily totals from RAW df

In [444]:
# Attempt to build Models

In [445]:
dfh1 = dfLoadHistory[['zone_id','date','Holiday','Weekend','h1']]
dfh1.head()


,zone_id,date,Holiday,Weekend,h1
0,1.0,2015-01-01,True,False,66025.0
1,2.0,2015-01-01,True,False,16908.0
2,3.0,2015-01-01,True,False,126314.0
3,4.0,2015-01-01,True,False,136288.0
4,5.0,2015-01-01,True,False,539.0


In [446]:
dfh1T = dfTempHist[['station_id','date','h1']]
dfh1T.head()

,station_id,date,h1
0,1,2015-01-01,45.0
1,2,2015-01-01,55.0
2,3,2015-01-01,47.0
3,4,2015-01-01,53.0
4,5,2015-01-01,54.0


In [480]:
def appendTemperature(dfTemp,dfHx,Hx):
    dfT = dfTempHist[['station_id','date',str(Hx)]]
    for x in range(1,10):
        dfinsertValues = dfT.loc[dfT['station_id']== str(x),[str(Hx),'date']]
        dfinsertValues.rename(columns = {str(Hx):'T' + str(x)}, inplace = True)
        dfHx = dfHx.merge(dfinsertValues,how='left', on=['date'])
    return dfHx

def buildDFdict(dfLoad, dfTemp):
    dfDic = {}
    for x in range(1,25):
        dfH = dfLoad[['zone_id','date','Holiday','Weekend','h'+str(x)]]
        dfDic['H'+str(x)]=appendTemperature(dfTemp,dfH,'h'+str(x))
    return dfDic

def prep4Model(df):
        df['date'] = pd.to_numeric(df['date'])
        df=pd.get_dummies(df,columns=['Holiday','Weekend'],drop_first=True)
        return df

def dataDicToExcel(dic):
    path="/Users/ryansupple/Desktop/ISDS 7075 Buisness Forecasting/Project/EnergyDictionary.xlsx"
    writer = pd.ExcelWriter(path, engine = 'xlsxwriter')
    for df in dic:
        dic[df].to_excel(writer, sheet_name=df)  
    

In [474]:
Testdic = buildDFdict(dfLoadHistory,dfTempHist)
dfTest = Testdic['H1']
print (dfTest)

      zone_id       date  Holiday  Weekend        h1    T1    T2    T3    T4  \
0         1.0 2015-01-01     True    False   66025.0  45.0  55.0  47.0  53.0   
1         2.0 2015-01-01     True    False   16908.0  45.0  55.0  47.0  53.0   
2         3.0 2015-01-01     True    False  126314.0  45.0  55.0  47.0  53.0   
3         4.0 2015-01-01     True    False  136288.0  45.0  55.0  47.0  53.0   
4         5.0 2015-01-01     True    False     539.0  45.0  55.0  47.0  53.0   
...       ...        ...      ...      ...       ...   ...   ...   ...   ...   
23650    11.0 2019-06-22    False     True   61989.0  73.0  81.0  73.0  78.0   
23651    12.0 2019-06-22    False     True   94692.0  73.0  81.0  73.0  78.0   
23652    13.0 2019-06-22    False     True  121706.0  73.0  81.0  73.0  78.0   
23653    14.0 2019-06-22    False     True   14546.0  73.0  81.0  73.0  78.0   
23654    15.0 2019-06-22    False     True   15810.0  73.0  81.0  73.0  78.0   

         T5    T6    T7    T8    T9  
0

In [481]:
dfTest = prep4Model(dfTest)
print(dfTest)

      zone_id                 date        h1    T1    T2    T3    T4    T5  \
0         1.0  1420070400000000000   66025.0  45.0  55.0  47.0  53.0  54.0   
1         2.0  1420070400000000000   16908.0  45.0  55.0  47.0  53.0  54.0   
2         3.0  1420070400000000000  126314.0  45.0  55.0  47.0  53.0  54.0   
3         4.0  1420070400000000000  136288.0  45.0  55.0  47.0  53.0  54.0   
4         5.0  1420070400000000000     539.0  45.0  55.0  47.0  53.0  54.0   
...       ...                  ...       ...   ...   ...   ...   ...   ...   
23650    11.0  1561161600000000000   61989.0  73.0  81.0  73.0  78.0  79.0   
23651    12.0  1561161600000000000   94692.0  73.0  81.0  73.0  78.0  79.0   
23652    13.0  1561161600000000000  121706.0  73.0  81.0  73.0  78.0  79.0   
23653    14.0  1561161600000000000   14546.0  73.0  81.0  73.0  78.0  79.0   
23654    15.0  1561161600000000000   15810.0  73.0  81.0  73.0  78.0  79.0   

         T6    T7    T8    T9  Holiday_True  Weekend_True  
0  

In [482]:
dfTestZ1 = dfTest[(dfTest.zone_id == "1.0")]
dfTestZ1.head()

,zone_id,date,h1,T1,T2,T3,T4,T5,T6,T7,T8,T9,Holiday_True,Weekend_True
0,1.0,1420070400000000000,66025.0,45.0,55.0,47.0,53.0,54.0,51.0,53.0,54.0,52.0,1,0
15,1.0,1420156800000000000,57097.0,49.0,52.0,54.0,41.0,51.0,45.0,55.0,50.0,54.0,0,0
30,1.0,1420243200000000000,58291.0,59.0,54.0,56.0,50.0,57.0,50.0,56.0,54.0,56.0,0,1
45,1.0,1420329600000000000,45723.0,71.0,72.0,70.0,66.0,71.0,67.0,70.0,72.0,70.0,0,1
60,1.0,1420416000000000000,40384.0,71.0,73.0,72.0,56.0,75.0,55.0,73.0,65.0,73.0,0,0


In [461]:
X = dfTestZ1[['date','T1', 'T2', 'T3', 'T4', 'T5', 'T6', 'T7', 'T8', 'T9', 'Holiday_True', 'Weekend_True']]
y = dfTestZ1[['h1']]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25)

,zone_id,date,Holiday,Weekend,h1,T1,T2,T3,T4,T5,T6,T7,T8,T9
0,1.0,2015-01-01,True,False,66025.0,45.0,55.0,47.0,53.0,54.0,51.0,53.0,54.0,52.0
1,2.0,2015-01-01,True,False,16908.0,45.0,55.0,47.0,53.0,54.0,51.0,53.0,54.0,52.0
2,3.0,2015-01-01,True,False,126314.0,45.0,55.0,47.0,53.0,54.0,51.0,53.0,54.0,52.0
3,4.0,2015-01-01,True,False,136288.0,45.0,55.0,47.0,53.0,54.0,51.0,53.0,54.0,52.0
4,5.0,2015-01-01,True,False,539.0,45.0,55.0,47.0,53.0,54.0,51.0,53.0,54.0,52.0
5,6.0,2015-01-01,True,False,6884.0,45.0,55.0,47.0,53.0,54.0,51.0,53.0,54.0,52.0
6,7.0,2015-01-01,True,False,133143.0,45.0,55.0,47.0,53.0,54.0,51.0,53.0,54.0,52.0
7,8.0,2015-01-01,True,False,136288.0,45.0,55.0,47.0,53.0,54.0,51.0,53.0,54.0,52.0
8,9.0,2015-01-01,True,False,3179.0,45.0,55.0,47.0,53.0,54.0,51.0,53.0,54.0,52.0
9,10.0,2015-01-01,True,False,75298.0,45.0,55.0,47.0,53.0,54.0,51.0,53.0,54.0,52.0


In [465]:
Testdic = prep4Model(Testdic)
dfTest = Testdic['H1']
dfTest.head()

,zone_id,date,Holiday,Weekend,h1,T1,T2,T3,T4,T5,T6,T7,T8,T9
0,1.0,2015-01-01,True,False,66025.0,45.0,55.0,47.0,53.0,54.0,51.0,53.0,54.0,52.0
1,2.0,2015-01-01,True,False,16908.0,45.0,55.0,47.0,53.0,54.0,51.0,53.0,54.0,52.0
2,3.0,2015-01-01,True,False,126314.0,45.0,55.0,47.0,53.0,54.0,51.0,53.0,54.0,52.0
3,4.0,2015-01-01,True,False,136288.0,45.0,55.0,47.0,53.0,54.0,51.0,53.0,54.0,52.0
4,5.0,2015-01-01,True,False,539.0,45.0,55.0,47.0,53.0,54.0,51.0,53.0,54.0,52.0


In [387]:
dfTestZ1 = dfTest[(dfTest.zone_id == "1.0")]
dfTestZ1.head()

,zone_id,date,h1,T1,T2,T3,T4,T5,T6,T7,T8,T9,Holiday_True,Weekend_True
0,1.0,1420070400000000000,66025.0,45.0,55.0,47.0,53.0,54.0,51.0,53.0,54.0,52.0,1,0
15,1.0,1420156800000000000,57097.0,49.0,52.0,54.0,41.0,51.0,45.0,55.0,50.0,54.0,0,0
30,1.0,1420243200000000000,58291.0,59.0,54.0,56.0,50.0,57.0,50.0,56.0,54.0,56.0,0,1
45,1.0,1420329600000000000,45723.0,71.0,72.0,70.0,66.0,71.0,67.0,70.0,72.0,70.0,0,1
60,1.0,1420416000000000000,40384.0,71.0,73.0,72.0,56.0,75.0,55.0,73.0,65.0,73.0,0,0


In [389]:
X = dfTestZ1[['date','T1', 'T2', 'T3', 'T4', 'T5', 'T6', 'T7', 'T8', 'T9', 'Holiday_True', 'Weekend_True']]
y = dfTestZ1[['h1']]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25)

regh1 = linear_model.LinearRegression()
regh1.fit(X_train, y_train)
print(regh1.score(X_test, y_test))

0.21583767415528154


In [390]:
print(regh1.coef_)

[[-7.32760888e-15 -3.77917723e+01 -3.75584131e+01 -3.67089695e+01
  -3.91156515e+01 -3.85051405e+01 -3.89997326e+01 -3.66616657e+01
  -3.94535097e+01 -3.87625020e+01  1.29372292e-02  1.17946592e-02]]


In [391]:
y_pred = regh1.predict(X_test)
print(y_pred)

[[60371.67622579]
 [59429.85101582]
 [63683.59434758]
 [52727.82291798]
 [47220.59311313]
 [60184.13302618]
 [66082.23706472]
 [66417.71317915]
 [58398.80227785]
 [51438.55488224]
 [49412.63107099]
 [61667.870024  ]
 [51092.18997639]
 [52895.7157275 ]
 [47878.43961362]
 [51864.04268762]
 [66907.49388741]
 [53936.23210462]
 [52320.54982074]
 [56441.27215908]
 [49248.32807934]
 [52395.19469333]
 [61983.88999628]
 [48151.02464195]
 [60499.62262448]
 [54483.72994209]
 [62558.76978127]
 [49322.54668849]
 [53842.59594453]
 [59859.55620632]
 [59333.86134279]
 [56824.67975192]
 [55687.88564079]
 [48908.36541575]
 [61562.02807009]
 [50166.42525575]
 [54361.10215673]
 [51516.9900748 ]
 [50308.96704551]
 [60179.60043911]
 [52904.917648  ]
 [56961.58156643]
 [59618.07510651]
 [49095.10568752]
 [48038.20456089]
 [47842.97929413]
 [49964.9840617 ]
 [46686.5750802 ]
 [55000.65621481]
 [62376.74855555]
 [57191.84292886]
 [53675.95753673]
 [45926.18245631]
 [50006.36912544]
 [64385.33354799]
 [52854.02

In [392]:
rmse = mean_squared_error(y_true=y_test,y_pred=y_pred,squared=False)

In [393]:
print(rmse)

10774.964802502718
